### let's start with the booking agent
Agents
1. **Flight Search Agent:**  Searches for available flights based on user preferences.
2. **Price Comparison Agent:**   Compares flight prices from different sources to get the best deal.
3. **Payment Processing Agent:** Handles payment and confirms the booking.
4. **Itinerary Manager:**  Sends booking details and manages modifications if needed.

In [1]:
from crewai import Agent,Task,Process,Crew
from crewai_tools import ScrapeWebsiteTool


c:\Users\tjwin\anaconda3\envs\gem_learn\lib\site-packages\pydantic\_internal\_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from key import gooogle_api_key

c:\Users\tjwin\anaconda3\envs\gem_learn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    verbose = True,
    temperature=0.5,
    google_api_key = gooogle_api_key
)

In [6]:
duckduck = ScrapeWebsiteTool("https://www.makemytrip.com")
from crewai_tools import SerperDevTool
from key import serp
import os
os.environ['SERPER_API_KEY'] =serp


serper = SerperDevTool()


search_agent = Agent(
    role = "Flight search agent",
    goal = "Search for flights tickets at minimum cost.",
    verbose = True,
    backstory = ("you are expert at searching in the flights at minimum cost."),
    tools = [duckduck,serper],
    llm = model
)

search_task = Task(
    description = (
        """search for the flight from the {origin} to {destination} on {date}.
        look for the best and cheap flights and return the links to book tickets."""
    ),
    expected_output = "a list of top flight booking links with prices and details",
    tools = [duckduck,serper],
    agent = search_agent
)


crew_flight = Crew(
    agents = [search_agent],
    tasks = [search_task],
    process = Process.sequential
)

result = crew_flight.kickoff(inputs = {
    "origin" :"Delhi",
    "destination" : "Mumbai",
    "date" : "2025-03-30"  
})

2025-02-19 19:04:52,061 - 13340 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
Thought: I need to search for flights from Delhi to Mumbai on March 30th, 2025, focusing on the cheapest options.  I'll use MakeMyTrip as a starting point.

Action: Read website content

Action Input: {"url": "https://www.makemytrip.com"}
 

Access Denied
Access Denied
You don't have permission to access "http://www.makemytrip.com/" on this server.
Reference #18.3e7056b8.1739972093.25fa8ffa
https://errors.edgesuite.net/18.3e7056b8.1739972093.25fa8ffa

Thought: MakeMyTrip is inaccessible. I need to try a different approach using a general web search.

Action: Search the internet

Action Input: {"search_query": "cheap flights Delhi to Mumbai March 30 2025"}
 


Search results: Title: Cheap Flights from Delhi to Mumbai (DEL-BOM) - Expedia.com
Link: https://www.expedia.com/lp/flights/del/bom/delhi-to-mumbai
Snippet: Cheap Flights from Delhi (DEL) to Mumbai (BOM) start at $55 for one-way and $111 for round trip. Earn your airline miles on top of o

In [7]:
print(result)

The following links may contain information on cheap flights from Delhi to Mumbai on March 30th, 2025.  Please note that I cannot access the content of these links to verify pricing and details.

* https://www.expedia.com/lp/flights/del/bom/delhi-to-mumbai
* https://www.kayak.com/flight-routes/New-Delhi-Indira-Gandhi-Intl-DEL/Mumbai-Chhatrapati-Shivaji-BOM
* https://www.skyscanner.com/routes/del/bom/indira-gandhi-international-to-mumbai.html
* https://www.makemytrip.com/flights/new_delhi-mumbai-cheap-airtickets.html
* https://www.momondo.com/flights/new-delhi/mumbai
* https://www.google.com/travel/flights/flights-from-new-delhi-to-mumbai.html
* https://www.makemytrip.com/international-flights/united_states/delhi-mumbai-cheap-airtickets.html
* https://www.goindigo.in/domestic-flights/delhi-to-mumbai-flights.html
* https://www.airindia.com/en-us/book-flights/delhi-to-mumbai-flights
* https://www.google.com/travel/flights/flights-to-mumbai.html


In [3]:
from crewai import  Crew,Agent,Task,Process
from key import gooogle_api_key,serp
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai_tools import SerperDevTool,ScrapeWebsiteTool
import os

In [ ]:
# Gemini flash model

model = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash",
    temperature = 0.5,
    verbose = True,
    google_api_key  = gooogle_api_key
)

#tool

os.environ["SERPER_API_KEY"] = serp
serper = SerperDevTool()
scraper = ScrapeWebsiteTool("https://www.makemytrip.com/")

# Search Agent

search = Agent(
    role = "Managing Speacialist",
    goal = "streamline the flights , trains ,hotels, and popular places at the destination.",
    backstory = ("""You are skilled in finding the best deals and securing bookings quickly at cheapest."""),
    memory = True,
    allow_deligation = True,
    tools = [serper,scraper],
    llm = model
)


#task


searching_task = Task(
    description = ("streamline the flight, and trains from {origin} to {destination} and hotels at{destination} for {people} people at cheapest and in budget of {budget} and search popular places in {destination}. they should be properly arranged and also give links in output."),
    #tools = [serper,scraper],
    agent = search,
    async_execution = True,
    expected_output = "blocks with in headings either flihgts, trains, hotels or places.",
    output_file = "links.md"
)


#crew

crew = Crew(
    agents = [search],
    task = [searching_task],
    process = Process.sequential
)

result = crew.kickoff(
    inputs = {
        "origin" :"Chandigarh",
        "destination" : "Srinagar",
        "people" : 3,
        "budget" : 30000,
    }
)


In [ ]:
from IPython.display import Markdown, display

# Path to your Markdown file


# Open and read the Markdown file
with open("links.md", "r", encoding="utf-8") as md_file:
    content = md_file.read()

# Render and display the Markdown content in the notebook
display(Markdown(content))